In [1]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
import schedulers

import tensorflow as tf
assert tf.config.list_physical_devices('GPU')

from tensorflow_addons.layers.crf import CRF
from tensorflow_addons.text.crf import crf_log_likelihood

In [ ]:

def unpack_data(data):
    if len(data) == 2:
        return data[0], data[1], None
    elif len(data) == 3:
        return data
    else:
        raise TypeError("Expected data to be a tuple of size 2 or 3.")


class ModelWithCRFLoss(keras.Model):
    """Wrapper around the base model for custom training logic."""

    def compute_loss(self, x, y, sample_weights, training=False):
        y_pred = self(x, training=training)
        # _, potentials, sequence_length, chain_kernel = y_pred
        potentials, sequence_length, chain_kernel = y_pred

        crf_loss = -crf_log_likelihood(potentials, y, sequence_length, chain_kernel)[0]

        if sample_weights is not None:
            crf_loss = crf_loss * sample_weights

        return tf.reduce_mean(crf_loss), sum(self.losses)

    def train_step(self, data):
        x, y, sample_weight = unpack_data(data)

        with tf.GradientTape() as tape:
            crf_loss, internal_losses = self.compute_loss(
                x, y, sample_weight, training=True
            )
            total_loss = crf_loss + internal_losses

        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return {"crf_loss": crf_loss, "internal_losses": internal_losses}

    def test_step(self, data):
        x, y, sample_weight = unpack_data(data)
        crf_loss, internal_losses = self.compute_loss(x, y, sample_weight)
        return {"crf_loss_val": crf_loss, "internal_losses_val": internal_losses}



In [2]:
BATCH_SIZE = 32

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)
KINDS_SIZE = len(dataset.KINDS)

def build_model(EMBED_DIM=50, UNITS=256):
    layer = input_text = keras.Input(batch_shape=(None, None), batch_size=BATCH_SIZE)
    layer = layers.Embedding(LETTERS_SIZE, EMBED_DIM, input_length=None, mask_zero=True)(layer)
    
    layer = layers.Dense(UNITS, activation=None)(layer)
    
    bidi = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.0), merge_mode='sum')
    layer = bidi(layer)
    layer = layers.concatenate([
        layers.add([layer, bidi(layer)]), 
        layers.subtract([layer, bidi(layer)])
    ])

    
#     outputs = [
#         CRF(NIQQUD_SIZE, name='N')(layer),
#         CRF(DAGESH_SIZE, name='D')(layer),
#         CRF(SIN_SIZE, name='S')(layer),
#     ]
#     model = ModelWithCRFLoss(inputs=input_text, outputs=outputs)
    
    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    model = keras.Model(inputs=input_text, outputs=outputs)
    # model.build((None, MAXLEN))

    jsmodel = model 
    # keras.utils.plot_model(model, to_file='model.png')
    return model, jsmodel

model, jsmodel = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     2200        input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, None, 256)    13056       embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, None, 256)    1050624     dense[0][0]                      
                                                                 bidirectional[0][0]          

In [3]:

def accuracy(y_true, y_pred):
    K = keras.backend
    f = K.floatx()
    # convert dense predictions to labels
    y_pred_labels =  K.cast(K.argmax(y_pred, axis=-1), f)
    
    res = K.cast(K.equal(y_true, y_pred_labels), f)
    return K.sum(res) / K.sum(K.cast(K.not_equal(y_true, 0), f))
    # return tf.gather(res, tf.where(K.not_equal(y_true, 0)))


In [4]:
def fit(train_validation, scheduler=None, verbose=1, lr=1e-4):
    train, valid = train_validation
    model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=lr), metrics=[accuracy])
    callbacks = []
    if isinstance(scheduler, schedulers.CircularLearningRate):
        scheduler.set_dataset(train, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)
        
    x  = train.normalized
    vx = valid.normalized
    
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin }
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin }
    
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=1, verbose=verbose, callbacks=callbacks)


In [5]:
MAXLEN = 82
def load_data(source, maxlen=MAXLEN, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid

In [6]:
data_other = load_data(['biblical', 'garbage', 'poetry'])

In [7]:
data_rabanit = load_data(['rabanit'])

In [8]:
data_pre_modern = load_data(['pre_modern'])

In [6]:
data_modern = load_data(validation=0.2, source=['modern'])

In [44]:
model.load_weights('./checkpoints/uninit')
history = fit(data_other, scheduler=schedulers.CircularLearningRate(30e-4, 150e-4, 5e-4))
model.save_weights('./checkpoints/other')

1210/1210 [==============================] - 61s 50ms/step - loss: 0.5488 - N_loss: 0.4344 - D_loss: 0.0965 - S_loss: 0.0178 - N_accuracy: 0.8511 - D_accuracy: 0.9624 - S_accuracy: 0.9944 - val_loss: 0.2637 - val_N_loss: 0.2117 - val_D_loss: 0.0479 - val_S_loss: 0.0041 - val_N_accuracy: 0.9293 - val_D_accuracy: 0.9818 - val_S_accuracy: 0.9984


In [45]:
model.load_weights('./checkpoints/other')
history = fit(data_rabanit, scheduler=schedulers.CircularLearningRate(30e-4, 50e-4, 5e-4))
model.save_weights('./checkpoints/rabanit')

1953/1953 [==============================] - 102s 52ms/step - loss: 0.1730 - N_loss: 0.1328 - D_loss: 0.0370 - S_loss: 0.0032 - N_accuracy: 0.9548 - D_accuracy: 0.9866 - S_accuracy: 0.9993 - val_loss: 0.2745 - val_N_loss: 0.2134 - val_D_loss: 0.0577 - val_S_loss: 0.0034 - val_N_accuracy: 0.9280 - val_D_accuracy: 0.9805 - val_S_accuracy: 0.9991


In [46]:
model.load_weights('./checkpoints/rabanit')
history = fit(data_pre_modern, scheduler=schedulers.CircularLearningRate(30e-4, 80e-4, 1e-4))
model.save_weights('./checkpoints/pre_modern')

1569/1569 [==============================] - 81s 52ms/step - loss: 0.2987 - N_loss: 0.2308 - D_loss: 0.0613 - S_loss: 0.0067 - N_accuracy: 0.9218 - D_accuracy: 0.9753 - S_accuracy: 0.9981 - val_loss: 0.2978 - val_N_loss: 0.2300 - val_D_loss: 0.0611 - val_S_loss: 0.0067 - val_N_accuracy: 0.9237 - val_D_accuracy: 0.9758 - val_S_accuracy: 0.9981


In [47]:
model.load_weights('./checkpoints/pre_modern')
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(5e-3, 6e-3, 6e-5))
model.save_weights('./checkpoints/modern')
model.save('models/modern.h5')

352/352 [==============================] - 30s 86ms/step - loss: 0.2193 - N_loss: 0.1661 - D_loss: 0.0499 - S_loss: 0.0033 - N_accuracy: 0.9436 - D_accuracy: 0.9811 - S_accuracy: 0.9991 - val_loss: 0.2296 - val_N_loss: 0.1739 - val_D_loss: 0.0521 - val_S_loss: 0.0036 - val_N_accuracy: 0.9433 - val_D_accuracy: 0.9802 - val_S_accuracy: 0.9991


In [16]:
data_mix = load_data(['poetry', 'rabanit', 'pre_modern'])

In [17]:
model.load_weights('./checkpoints/uninit')
history = fit(data_mix, scheduler=schedulers.CircularLearningRate(30e-4, 80e-4, 1e-4)) #  no biblical: 0.9360, no biblical, bo garbage: 0.9300
model.save_weights('./checkpoints/mix')

3368/3368 [==============================] - 211s 63ms/step - loss: 0.3123 - N_loss: 0.2413 - D_loss: 0.0627 - S_loss: 0.0083 - N_accuracy: 0.9163 - D_accuracy: 0.9756 - S_accuracy: 0.9977 - val_loss: 0.2714 - val_N_loss: 0.2115 - val_D_loss: 0.0540 - val_S_loss: 0.0058 - val_N_accuracy: 0.9300 - val_D_accuracy: 0.9792 - val_S_accuracy: 0.9984


In [18]:
model.load_weights('./checkpoints/mix')
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(5e-3, 6e-3, 6e-5))  # no biblical: 0.9593, no biblical, bo garbage: 0.9593 
model.save_weights('./checkpoints/modern')

351/351 [==============================] - 34s 96ms/step - loss: 0.1952 - N_loss: 0.1464 - D_loss: 0.0458 - S_loss: 0.0030 - N_accuracy: 0.9507 - D_accuracy: 0.9829 - S_accuracy: 0.9993 - val_loss: 0.1690 - val_N_loss: 0.1238 - val_D_loss: 0.0419 - val_S_loss: 0.0032 - val_N_accuracy: 0.9593 - val_D_accuracy: 0.9845 - val_S_accuracy: 0.9993


In [ ]:
for i in range(50):
    model.load_weights('./checkpoints/pre_modern')
    p1 = np.exp(np.random.uniform(low=np.log(1e-5), high=np.log(1e-2)))
    p2 = np.exp(np.random.uniform(low=np.log(1e-4), high=np.log(1e-1)))
    p3 = np.exp(np.random.uniform(low=np.log(1e-5), high=np.log(1e-2)))
    print(p1, p2, p3, end=', ', sep=', ')
    history = fit(data_modern, scheduler=schedulers.CircularLearningRate(p1, p2, p3), verbose=0)
    print(history.history['val_N_accuracy'][0])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [19]:

model.save('models/modern.h5')
import tensorflowjs as tfjs
jsmodel.load_weights('./checkpoints/modern')
tfjs.converters.save_keras_model(jsmodel, '.')

c:\users\elaza\appdata\local\programs\python\python38\lib\site-packages\tensorflowjs\converters\keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [23]:
model.load_weights('./checkpoints/modern')

def print_predictions(data, s):
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud[s], data.dagesh[s], data.sin[s]]
    actual = dataset.merge(data.text[s], ts=batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(data.text[s], ts=batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    total = []
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)
        last = expected_niqqud[i].tolist().index(0)
        res = expected_niqqud[i][:last] == actual_niqqud[i][:last]
        total.extend(res)
        print(round(np.mean(res), 2), f'({last - sum(res)} out of {last})')
        print()
    print(round(np.mean(total), 3))

print_predictions(data_modern[1], slice(0, None))

מצוי:  הַסִּיבָּה שֶׁבִּגְלָלָה לֹא רָצִיתִי יוֹתֵר אֶת הַקֶּשֶׁר הַזֶּה. לֹא רָצִיתִי אֶת הַהֲמוּלָה הַזֹּאת וְשֶׁכּוּלָם יִדַבְּרוּ עַל 
רצוי:  הַסִּיבָּה שֶׁבִּגְלָלָהּ לֹא רָצִיתִי יוֹתֵר אֶת הַקֶּשֶׁר הַזֶּה. לֹא רָצִיתִי אֶת הַהֲמוּלָּה הַזֹּאת וְשֶׁכּוּלָּם יְדַבְּרוּ עַל 
0.99 (1 out of 80)

מצוי:  שֶׁאֶת הַמִּשְׁפָּט, "אִם הִרְחַקְתִּי לִרְאוֹת, הֲרֵי זֶה מִשּׁוּם שֶׁעָמַדְתִּי עַל כִּתְפֵיהֶם שֶׁל עֲנָקִים", כָּתַב נִיוּטוֹן 
רצוי:  שֶׁאֶת הַמִּשְׁפָּט, "אִם הִרְחַקְתִּי לִרְאוֹת, הֲרֵי זֶה מִשּׁוּם שֶׁעָמַדְתִּי עַל כִּתְפֵיהֶם שֶׁל עֲנָקִים", כָּתַב נְיוּטוֹן 
0.99 (1 out of 80)

מצוי:  וּבְטִיחוּתִית, כָּל נַעַר מִתְבַּגֵּר אוֹ בּוֹגֵר צָעִיר בְּנַפְשׁוֹ יֶהֲנֶה מְחֲוּויָה מְאַתְגֶּרֶת מְלֵאַת אֶקְשָׁן. אַחֲרֵי 
רצוי:  וּבְטִיחוּתִית, כָּל נַעַר מִתְבַּגֵּר אוֹ בּוֹגֵר צָעִיר בְּנַפְשׁוֹ יֵהָנֶה מֵחֲוָויָה מְאַתְגֶּרֶת מְלֵאַת אֶקְשֵׁן. אַחֲרֵי 
0.94 (5 out of 78)

מצוי:  הַשִּׁיטוֹת שֶׁקָּדְמוּ לָהּ. קְבִיעָתוֹ שֶׁל נִיוּטוֹן שֶׁהַיְּקוּם כּוּלּוֹ כָּפוּף לְעֶקְרוֹנוֹת מִ

In [ ]:
shutil.rmtree(os.sep.join([tempfile.gettempdir(), '.tensorboard-info']), ignore_errors=True)
shutil.rmtree('logs', ignore_errors=True)
os.makedirs('logs')
# %tensorboard --logdir logs

In [ ]:
print(data_modern[1].text[0])
print(data_modern[1].text[1])

In [ ]:
[hex(ord(x)) for x in 'כָּ']